In [1]:
import csv
import random
import re
from pathlib import Path
from xml.etree.ElementTree import ElementTree
from tqdm import tqdm
import nltk

# Téléchargement de WordNet si nécessaire
try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [2]:
# Constantes pour WordNet
WORDNET_POS = {'VERB': wn.VERB, 'NOUN': wn.NOUN, 'ADJ': wn.ADJ, 'ADV': wn.ADV}

# Constantes globales
HEADERS = ['id', 'sentence', 'sense_keys', 'glosses', 'targets']
TGT_TOKEN = '[TGT]'
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

# Paramètres utilisateur à définir directement
corpus_dir = "/content/corpus_dir"  # Répertoire contenant les fichiers XML et TXT
output_dir = "/content/output_dir"  # Répertoire où enregistrer le fichier CSV
max_num_gloss = 5  # Limite max des gloses (mettre une valeur ou None pour aucune limite)
use_augmentation = True  # Activer l'augmentation des données avec WordNet

In [3]:
def _get_info(lemma, pos, info_type):
    """
    Récupère les informations de WordNet pour un lemme donné, comme les gloses ou les exemples.
    """
    results = dict()
    wn_pos = WORDNET_POS[pos] if pos is not None else None
    morphemes = wn._morphy(lemma, pos=wn_pos) if pos is not None else []
    for i, synset in enumerate(set(wn.synsets(lemma, pos=wn_pos))):
        sense_key = None
        for l in synset.lemmas():
            if l.name().lower() == lemma.lower():
                sense_key = l.key()
                break
            elif l.name().lower() in morphemes:
                sense_key = l.key()
        assert sense_key is not None
        results[sense_key] = synset.examples() if info_type == 'examples' else synset.definition()
    return results

def get_glosses(lemma, pos):
    """
    Récupère les gloses (définitions) associées à un lemme et une catégorie grammaticale.
    """
    return _get_info(lemma, pos, info_type='gloss')

def get_example_sentences(lemma, pos):
    """
    Récupère les phrases d'exemple associées à un lemme et une catégorie grammaticale.
    """
    return _get_info(lemma, pos, info_type='examples')

def get_all_wordnet_lemma_names():
    """
    Récupère tous les lemmes de WordNet classés par catégorie grammaticale.
    """
    results = []
    for pos, wn_pos in WORDNET_POS.items():
        results.append((pos, wn.all_lemma_names(pos=wn_pos)))
    return results

In [7]:
def generate_csv():
    """
    Génère un fichier CSV à partir d'un corpus XML/WordNet pour une tâche de sélection de gloses.
    """
    corpus_dir_path = Path(corpus_dir)
    corpus_name = corpus_dir_path.name.lower()
    xml_path = str(corpus_dir_path.joinpath(f"{corpus_name}.data.xml"))
    txt_path = str(corpus_dir_path.joinpath(f"{corpus_name}.gold.key.txt"))
    output_filename = f"{corpus_name}"
    if max_num_gloss:
        output_filename += f"-max_num_gloss={max_num_gloss}"
    if use_augmentation:
        output_filename += "-augmented"
    csv_path = str(Path(output_dir).joinpath(f"{output_filename}.csv"))

    print("Création des données pour la tâche de sélection de gloses...")
    record_count = 0
    gloss_count = 0
    max_gloss_count = 0

    # Traitement du fichier XML et génération du CSV
    xml_root = ElementTree(file=xml_path).getroot()
    with open(csv_path, 'w', encoding='utf-8', newline='') as f:
        csv_writer = csv.writer(f)
        csv_writer.writerow(HEADERS)

        def _write_to_csv(_id, _sentence, _lemma, _pos, _gold_keys):
            """
            Écrit une ligne dans le fichier CSV.
            """
            nonlocal record_count, gloss_count, max_gloss_count

            # Récupération des gloses pour le lemme donné
            sense_info = get_glosses(_lemma, _pos)
            if max_num_gloss is not None:
                sense_gloss_pairs = []
                for k in _gold_keys:
                    sense_gloss_pairs.append((k, sense_info[k]))
                    del sense_info[k]

                remainder = max_num_gloss - len(sense_gloss_pairs)
                if len(sense_info) > remainder:
                    for p in random.sample(list(sense_info.items()), remainder):
                        sense_gloss_pairs.append(p)
                elif len(sense_info) > 0:
                    sense_gloss_pairs += list(sense_info.items())

                random.shuffle(sense_gloss_pairs)
                sense_keys, glosses = zip(*sense_gloss_pairs)
            else:
                sense_keys, glosses = zip(*sense_info.items())

            # Création des cibles et écriture dans le CSV
            targets = [sense_keys.index(k) for k in _gold_keys]
            csv_writer.writerow([_id, _sentence, list(sense_keys), list(glosses), targets])

            record_count += 1
            gloss_count += len(glosses)
            max_gloss_count = max(max_gloss_count, len(glosses))

        with open(txt_path, 'r', encoding='utf-8') as g:
            for doc in tqdm(xml_root):
                for sent in doc:
                    tokens = []
                    instances = []
                    for token in sent:
                        tokens.append(token.text)
                        if token.tag == 'instance':
                            start_idx = len(tokens) - 1
                            end_idx = start_idx + 1
                            instances.append((token.attrib['id'], start_idx, end_idx, token.attrib['lemma'], token.attrib['pos']))

                    for id_, start, end, lemma, pos in instances:
                        gold = g.readline().strip().split()
                        gold_keys = gold[1:]
                        assert id_ == gold[0]

                        sentence = " ".join(
                            tokens[:start] + [TGT_TOKEN] + tokens[start:end] + [TGT_TOKEN] + tokens[end:]
                        )
                        _write_to_csv(id_, sentence, lemma, pos, gold_keys)

        if use_augmentation:
            print("Création de données supplémentaires en utilisant les phrases d'exemple de WordNet...")
            counter = 0
            for pos, lemma_name_generator in get_all_wordnet_lemma_names():
                print(f"Traitement de {pos}...")
                for lemma in tqdm(list(lemma_name_generator)):
                    for gold_key, examples in get_example_sentences(lemma, pos).items():
                        for example_sentence in examples:
                            re_result = re.search(rf"\b{lemma.lower()}\b", example_sentence.lower())
                            if re_result is not None:
                                start, end = re_result.span()
                                sentence = f"{example_sentence[:start]}" \
                                    f"{TGT_TOKEN} {example_sentence[start:end]} {TGT_TOKEN}" \
                                    f"{example_sentence[end:]}".strip()
                                _write_to_csv(f"wn-aug-{counter}", sentence, lemma, pos, [gold_key])
                                counter += 1

    print(
        f"Terminé.\n"
        f"Nombre d'enregistrements : {record_count}\n"
        f"Nombre moyen de gloses par enregistrement : {gloss_count / record_count:.2f}\n"
        f"Nombre maximum de gloses dans un enregistrement : {max_gloss_count}"
    )

In [6]:
# Exécution de la fonction
generate_csv()

Création des données pour la tâche de sélection de gloses...


  0%|          | 0/352 [00:00<?, ?it/s]<ipython-input-4-44d56ced9f8c>:43: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  for p in random.sample(sense_info.items(), remainder):
100%|██████████| 352/352 [00:21<00:00, 16.43it/s]


Création de données supplémentaires en utilisant les phrases d'exemple de WordNet...
Traitement de VERB...


100%|██████████| 11529/11529 [00:01<00:00, 7551.56it/s]


Traitement de NOUN...


...

Traitement de ADJ...


100%|██████████| 21479/21479 [00:02<00:00, 7836.05it/s]


Traitement de ADV...


100%|██████████| 4481/4481 [00:00<00:00, 7750.88it/s]


Terminé.
Nombre d'enregistrements : 263632
Nombre moyen de gloses par enregistrement : 3.51
Nombre maximum de gloses dans un enregistrement : 5


In [9]:
# Exécution de la fonction
generate_csv()

Création des données pour la tâche de sélection de gloses...


100%|██████████| 3/3 [00:00<00:00, 72.45it/s]


Création de données supplémentaires en utilisant les phrases d'exemple de WordNet...
Traitement de VERB...


100%|██████████| 11529/11529 [00:01<00:00, 9981.91it/s] 


Traitement de NOUN...


...

Traitement de ADJ...


100%|██████████| 21479/21479 [00:02<00:00, 9696.96it/s] 


Traitement de ADV...


100%|██████████| 4481/4481 [00:00<00:00, 10095.22it/s]

Terminé.
Nombre d'enregistrements : 38051
Nombre moyen de gloses par enregistrement : 2.86
Nombre maximum de gloses dans un enregistrement : 5
